# Multireference Perturbation theory equations with symbolic algebra

In this notebook we will show that the multireference perturbation theory (more specifically, NEVPT) equations can be automatically derived with the use of symbolic algebra. Some knowledge about quantum chemistry and python are expected ; this notebook assumes for the larger part completion of the coupled-cluster hands-on session.

Make sure that you have the following installed:

* Python 2 (this notebook **will not work** with Python 3)
* IPython/Jupyter

Misc imports to add at the beginning of a script:

In [1]:
import warnings
warnings.filterwarnings('ignore')

***

## Background

 (Note that we use Einstein's notation: the summations of repeated indexes are implicit, i.e. $A_{ab}=\sum_{cd} B_{abcd}$ is just written $A_{ab}=B_{abcd}$.)

### Multireference perturbation theory equations and notations

We wish to obtain the second and third order correction to the energy:
$$
\begin{array}{rl}
E_2&=\left\langle \Psi_0\middle|\;\hat{V}\;\middle|\Psi_1\right\rangle
\\
E_3&=\left\langle \Psi_1\middle|\;\hat{V}-E_1\;\middle|\Psi_1\right\rangle
\end{array}
$$

where the Hamiltonian has been split as:
$$\hat{H}=\hat{H}_0+\hat{V}$$

and where $\left|\Psi_0\right\rangle$ has been obtained by a zero-th order CAS-like scheme, and is composed of core, active and virtual orbitals.

The first-order correction to the wavefunction, $\left|\Psi_1\right\rangle$, is found as the minimum of the Hylleraas functional:
$$\left\langle\Psi_1\middle|\;\hat{H}_0-E_0\;\middle|\Psi_1\right\rangle
+\left\langle\Psi_1\middle|\;\hat{V}\;\middle|\Psi_0\right\rangle=0$$

i.e. by solving:
$$\left(E_0-\hat{H}_0\right)\left|\Psi_1\right\rangle=\hat{V}\left|\Psi_0\right\rangle\qquad\qquad(1)$$

### Internally contracted MRPT

In internally contracted MRPT, we are looking for the first-order correction to the wavefunction, $\left|\Psi_1\right\rangle$ expanded on a basis of perturber wavefunctions **that are connected to $\left|\Psi_0\right\rangle$, i.e. that are excitations of $\left|\Psi_0\right\rangle$:
$$\left|\Psi_1\right\rangle=d_\mu \hat{E}_\mu\left|\Psi_0\right\rangle$$

Given that
$$\hat{V}\left|\Psi_0\right\rangle=w_\tau\hat{E}_\tau\left|\Psi_0\right\rangle$$

Equation (1) amounts to solving the following equation for the coefficients $d_\mu$:
$$\left\langle\Psi_0\middle|\; \hat{E}_\mu \left(E_0-\hat{H}_0\right) \hat{E}_\nu \;\middle|\Psi_0\right\rangle d_\nu
 =\left\langle\Psi_0\middle|\; \hat{E}_\mu \hat{E}_\tau \;\middle|\Psi_0\right\rangle w_\tau
$$

At this point a trick is used: the realization that $\left(E_0-\hat{H}_0\right)\left|\Psi_0\right\rangle=0$ allows the introduction of a commutator, and the equation becomes:
$$\left\langle\Psi_0\middle|\; \hat{E}_\mu \left[\left(E_0-\hat{H}_0\right),\hat{E}_\nu\right] \;\middle|\Psi_0\right\rangle d_\nu
 =\left\langle\Psi_0\middle|\; \hat{E}_\mu \hat{E}_\tau \;\middle|\Psi_0\right\rangle w_\tau
 \quad \iff \quad A_{\mu\nu}d_\nu=S_{\mu\tau}w_\tau
 \qquad\qquad(2)$$

Basically the terms to manipulate are long strings of creation/annihiltion operators,
and one needs to use the Wick’s theorem to simplify the expectation values.
The resulting expressions will be series of tensors involving one- and two-electron integrals, and RDMs up to four-order.

You can convince yourself (**ACTIVITY?**) that when dealing with a wavefunction having core, active and virtual orbitals there are 8 classes of exictations on $\left|\Psi_0\right\rangle$ that are connected to $\left|\Psi_0\right\rangle$. There are usually named after their pattern of change of occupation of the core, active and virtual spaces with respect to $\left|\Psi_0\right\rangle$:

| #     | Name  | core| act | virt| operator |
|-------|-------|-----|-----|-----|---|
| I     | CCVV  | -2  |  0  |  2  | $\hat{E}_i^a\hat{E}_j^b$ |
| II    | ACVV  | -1  | -1  |  2  | $\hat{E}_i^a\hat{E}_p^b$ |
| III   | CCAV  | -2  |  1  |  1  | $\hat{E}_i^a\hat{E}_j^p$ |
| IV    | AAVV  |  0  | -2  |  2  | $\hat{E}_p^a\hat{E}_q^b$ |
| V     | CCAA  | -2  |  2  |  0  | $\hat{E}_i^p\hat{E}_j^q$ |
| VI    | CAAV  | -1  |  0  |  1  | $\hat{E}_i^a\hat{E}_p^q, \hat{E}_q^a\hat{E}_i^p$ |
| VII   | AAAC  | -1  |  1  |  0  | $\hat{E}_p^q\hat{E}_i^r$ |
| VIII  | AAAV  |  0  | -1  |  1  | $\hat{E}_p^q\hat{E}_r^a$ |

which is projected unto the HF and the excitated basis of the operator $\hat{T}$:
$$\left\{
\begin{array}{lr}
&\left\langle 0\middle|e^{-\hat{T}}\hat{H}e^{\hat{T}}\middle|0\right\rangle=E
\\
\;\forall\mu
&C_\mu=\left\langle 0\middle|\hat{E}_\mu \; e^{-\hat{T}}\hat{H}e^{\hat{T}}\middle|0\right\rangle=0
\end{array}
\right.\qquad\qquad(1)$$

The second equation is to be solved for the amplitudes $t_\mu$ and the first equation gives the energy from those amplitudes.

### Tensors and excitation operators

The Hamiltonian $\hat{H}$ and the operator $\hat{T}$ are composed of **tensors** and **excitation operators**:
$$\hat{H}=v_p^q \hat{E}_p^q + \frac{1}{2} v_{pq}^{rs} \hat{E}_{pq}^{rs}\qquad\qquad(A)$$

and for example in the case of CCD:
$$\hat{T}=t_{ij}^{ab}\hat{E}_{ij}^{ab}\qquad\qquad(B)$$

so that any expectation value seen in Equation (1) will reduce to contractions of integrals $v$, amplitudes $t$ and reduced density matrices $R$, as examplify in the mock expectation value below:
$$\left\langle 0\middle| 
\left(v_{cd}^{kl} \hat{E}_{cd}^{kl}\right)
\left(t_{ij}^{ab} \hat{E}_{ij}^{ab}\right) 
\middle|0\right\rangle
=v_{cd}^{kl}t_{ij}^{ab}\left\langle 0\middle| \hat{E}_{cd}^{kl} \hat{E}_{ij}^{ab} \middle|0\right\rangle
=v_{cd}^{kl}t_{ij}^{ab} R_{ijkl,abcd}$$

In practice the reduced-density matrices will only yield delta functions on the indexes involved.

### Normal-ordering

>**<span style="color:blue">What is at stake here is the lenghtly derivation of the normal ordering (Wick's theorem) involved to transform the expectation values in Equation (1) into tractable and computable objects.</span>**

In the mock expectation value shown above, we encounter the term
$$\left\langle 0\middle| \hat{E}_{cd}^{kl} \hat{E}_{ij}^{ab} \middle|0\right\rangle
=\left\langle 0\middle| a^\dagger_c a^\dagger_d a_l a_k a^\dagger_i a^\dagger_j a_b a_a  \middle|0\right\rangle
$$

which is not the optimal expression for this particular string of operators. If you instead transform it using the anticommutation relations into a string where all creation operators are on the left and all destruction operators are on the right, as in :
$$\left\langle 0\middle| a^\dagger_c a^\dagger_d a^\dagger_i a^\dagger_j a_l a_k a_b a_a  \middle|0\right\rangle$$

This particular expression is easy to analyze and compute because it easily leads to zeros and deltas (think about the applications of the creation and annihilation operators to $\left|0\right\rangle$).

Wick's theorem states that any arbitrary string of operators (a encountered when dealing with expectation values) can be written as a linear combination of normal-ordered strings. 

**Hence, applying Wick's theorem to a string of operators yields a sum of easy-to-compute terms.**

Although lenghtly and prone to error, the derivation of the normal ordering of the operators is a very systematic task that can easily be done automatically. One only needs to teach a program the basics of the algebra (the anticommutation relations) to allow it to derive lenghly equations seamlessly. This is called **symbolic algebra**, and a good library to do that in **Second Quantization Algebra** (`sqa`).

***

## First steps in Second Quantization Algebra

Let's import the `sqa` library (see installation instructions):

In [2]:
import secondQuantizationAlgebra as sqa
from tools_for_sqa import simplify_all, is_non_zero, list_indexes, list_tensors
sqa.options.verbose = False

The script needs to know what indexes refer to occupied or virtual orbitals, this is done by defining "tags":

In [3]:
tag_occupied= sqa.options.core_type
tag_virtual = sqa.options.virtual_type

Then we will define objects that `sqa` is going to manipulate. Those are:
* indexes (occupied or virtual)
* tensors ($v$ and $t$)
* excitation operators ($\hat{E}$)
* `terms`: in `sqa`, a term is a constant + a tensor + an excitation operator

Note that `sqa` does not have documentation, but the code is well commented, so this notebook sometimes refer to lines of code that are comments (this assumes the use of the fork from `github.com/mussard`).

***

## Define the Hamiltonian (A)

Remember that the Hamiltonian is defined as:
$$\hat{H}=v_p^q \hat{E}_p^q + \frac{1}{2} v_{pq}^{rs} \hat{E}_{pq}^{rs}\qquad\qquad(A)$$

### A\ First, let's define a bunch of indexes

The librairy `sqa` is going to need to distinguish between indexes of occupied and of virtual orbitals (this will impact the anticommutation relations and will allow simplification in the expressions, since for example application of a creation operator on an occupied index yields a zero, etc...).

The **indexes** in `sqa` are defined as follow:

In [4]:
iH  = sqa.index('iH', [tag_occupied], True)
jH  = sqa.index('jH', [tag_occupied], True)
kH  = sqa.index('kH', [tag_occupied], True)
lH  = sqa.index('lH', [tag_occupied], True)
aH  = sqa.index('aH', [tag_virtual],  True)
bH  = sqa.index('bH', [tag_virtual],  True)
cH  = sqa.index('cH', [tag_virtual],  True)
dH  = sqa.index('dH', [tag_virtual],  True)

This is nothing more than a human-readable name (a string) and a tag that specifies the occupied or virtual nature of the index (there is also a dummy attribute).

### B\ Then, we define the Hamiltonian tensors

The **tensors** in `sqa` have again a human-readable name (a string) and a list of indexes (they also have symmetry property that we overlook for now):

In [5]:
Voo  =sqa.tensor('Voo',   [iH,jH]      , []) # this is Voo(i,j)
Vov  =sqa.tensor('Vov',   [iH,aH]      , [])
Vvo  =sqa.tensor('Vvo',   [aH,iH]      , [])
Vvv  =sqa.tensor('Vvv',   [aH,bH]      , [])
Voooo=sqa.tensor('Voooo', [iH,jH,kH,lH], []) # this is Voooo(i,j,k,l)
Vooov=sqa.tensor('Vooov', [iH,jH,kH,aH], [])
Voovo=sqa.tensor('Voovo', [iH,jH,aH,kH], [])
Voovv=sqa.tensor('Voovv', [iH,jH,aH,bH], [])
Vovoo=sqa.tensor('Vovoo', [iH,aH,jH,kH], [])
Vovov=sqa.tensor('Vovov', [iH,aH,jH,bH], [])
Vovvo=sqa.tensor('Vovvo', [iH,aH,bH,jH], [])
Vovvv=sqa.tensor('Vovvv', [iH,aH,bH,cH], [])
Vvooo=sqa.tensor('Vvooo', [aH,iH,jH,kH], [])
Vvoov=sqa.tensor('Vvoov', [aH,iH,jH,bH], [])
Vvovo=sqa.tensor('Vvovo', [aH,iH,bH,jH], [])
Vvovv=sqa.tensor('Vvovv', [aH,iH,bH,cH], [])
Vvvoo=sqa.tensor('Vvvoo', [aH,bH,iH,jH], [])
Vvvov=sqa.tensor('Vvvov', [aH,bH,iH,cH], [])
Vvvvo=sqa.tensor('Vvvvo', [aH,bH,cH,iH], [])
Vvvvv=sqa.tensor('Vvvvv', [aH,bH,cH,dH], [])

(See for example lines 23-42 of the `sqaTensor.py` file for more context.)

### C\ We need to define And the excitation operators for the Hamiltonian

The **excitation operators** in `sqa` are just a list of indexes that are creation and annihilation operators:

In [6]:
Eoo  =sqa.sfExOp([iH,jH]      ) # this is a^dagger_i(1) a_j(2)
Eov  =sqa.sfExOp([iH,aH]      )
Evo  =sqa.sfExOp([aH,iH]      )
Evv  =sqa.sfExOp([aH,bH]      )
Eoooo=sqa.sfExOp([iH,jH,kH,lH]) # this is a^dagger_i(1) a^dagger_j(2) a_l(1) a_k(2)
Eooov=sqa.sfExOp([iH,jH,kH,aH])
Eoovo=sqa.sfExOp([iH,jH,aH,kH])
Eoovv=sqa.sfExOp([iH,jH,aH,bH])
Eovoo=sqa.sfExOp([iH,aH,jH,kH])
Eovov=sqa.sfExOp([iH,aH,jH,bH])
Eovvo=sqa.sfExOp([iH,aH,bH,jH])
Eovvv=sqa.sfExOp([iH,aH,bH,cH])
Evooo=sqa.sfExOp([aH,iH,jH,kH])
Evoov=sqa.sfExOp([aH,iH,jH,bH])
Evovo=sqa.sfExOp([aH,iH,bH,jH])
Evovv=sqa.sfExOp([aH,iH,bH,cH])
Evvoo=sqa.sfExOp([aH,bH,iH,jH])
Evvov=sqa.sfExOp([aH,bH,iH,cH])
Evvvo=sqa.sfExOp([aH,bH,cH,iH])
Evvvv=sqa.sfExOp([aH,bH,cH,dH])

(The object `sqa.sfExOp` is a child of the tensor object : see for example lines 346-348 of the `sqaTensor.py` file for more context.)

### D\ Finally, we define the terms of the Hamiltonian

Remember again that the Hamiltonian is defined as:
$$\hat{H}=v_p^q \hat{E}_p^q + \frac{1}{2} v_{pq}^{rs} \hat{E}_{pq}^{rs}\qquad\qquad(A)$$

We define the hamiltonian as a list of **terms**, which in `sqa` have a numerical constant (here 1 or 1/2), something else that is not important here, and a list of tensors and excitation operators:

In [7]:
H=[
    sqa.term(1.0, [''], [Voo   ,Eoo  ]), # this is Voo(i,j) E_j^i
    sqa.term(1.0, [''], [Vov   ,Eov  ]),
    sqa.term(1.0, [''], [Vvo   ,Evo  ]),
    sqa.term(1.0, [''], [Vvv   ,Evv  ]),
    sqa.term(0.5, [''], [Voooo ,Eoooo]), # this is 1/2 Voooo(ijkl) E_{kl}^{ij}
    sqa.term(0.5, [''], [Vooov ,Eooov]),
    sqa.term(0.5, [''], [Voovo ,Eoovo]),
    sqa.term(0.5, [''], [Voovv ,Eoovv]),
    sqa.term(0.5, [''], [Vovoo ,Eovoo]),
    sqa.term(0.5, [''], [Vovov ,Eovov]),
    sqa.term(0.5, [''], [Vovvo ,Eovvo]),
    sqa.term(0.5, [''], [Vovvv ,Eovvv]),
    sqa.term(0.5, [''], [Vvooo ,Evooo]),
    sqa.term(0.5, [''], [Vvoov ,Evoov]),
    sqa.term(0.5, [''], [Vvovo ,Evovo]),
    sqa.term(0.5, [''], [Vvovv ,Evovv]),
    sqa.term(0.5, [''], [Vvvoo ,Evvoo]),
    sqa.term(0.5, [''], [Vvvov ,Evvov]),
    sqa.term(0.5, [''], [Vvvvo ,Evvvo]),
    sqa.term(0.5, [''], [Vvvvv ,Evvvv])
]

In [8]:
print(len(H))
for elt in H:
    print(elt)

20
 (   1.00000)  Voo[iH,jH] E1[iH,jH] 
 (   1.00000)  Vov[iH,aH] E1[iH,aH] 
 (   1.00000)  Vvo[aH,iH] E1[aH,iH] 
 (   1.00000)  Vvv[aH,bH] E1[aH,bH] 
 (   0.50000)  Voooo[iH,jH,kH,lH] E2[iH,jH,kH,lH] 
 (   0.50000)  Vooov[iH,jH,kH,aH] E2[iH,jH,kH,aH] 
 (   0.50000)  Voovo[iH,jH,aH,kH] E2[iH,jH,aH,kH] 
 (   0.50000)  Voovv[iH,jH,aH,bH] E2[iH,jH,aH,bH] 
 (   0.50000)  Vovoo[iH,aH,jH,kH] E2[iH,aH,jH,kH] 
 (   0.50000)  Vovov[iH,aH,jH,bH] E2[iH,aH,jH,bH] 
 (   0.50000)  Vovvo[iH,aH,bH,jH] E2[iH,aH,bH,jH] 
 (   0.50000)  Vovvv[iH,aH,bH,cH] E2[iH,aH,bH,cH] 
 (   0.50000)  Vvooo[aH,iH,jH,kH] E2[aH,iH,jH,kH] 
 (   0.50000)  Vvoov[aH,iH,jH,bH] E2[aH,iH,jH,bH] 
 (   0.50000)  Vvovo[aH,iH,bH,jH] E2[aH,iH,bH,jH] 
 (   0.50000)  Vvovv[aH,iH,bH,cH] E2[aH,iH,bH,cH] 
 (   0.50000)  Vvvoo[aH,bH,iH,jH] E2[aH,bH,iH,jH] 
 (   0.50000)  Vvvov[aH,bH,iH,cH] E2[aH,bH,iH,cH] 
 (   0.50000)  Vvvvo[aH,bH,cH,iH] E2[aH,bH,cH,iH] 
 (   0.50000)  Vvvvv[aH,bH,cH,dH] E2[aH,bH,cH,dH] 


(See for example lines 26-31 of the `sqaTerm.py` file for more (limited) context.)

***

## Define the $\hat{T}$ operators (B)

Remember the definition of the $\hat{T}$ operator in the case of CCD:
$$\hat{T}=t_{ij}^{ab}\hat{E}_{ij}^{ab}\qquad\qquad(B)$$

We are going to follow the same steps as for the Hamiltonian

### We define the indexes, tensor, operator, and term for $\hat{T}$ and $\hat{T}^2$

We need new indexes (note the "T" where before there was "H" in the variable name):

In [9]:
iT = sqa.index('iT', [tag_occupied], True)
jT = sqa.index('jT', [tag_occupied], True)
kT = sqa.index('kT', [tag_occupied], True)
lT = sqa.index('lT', [tag_occupied], True)
aT = sqa.index('aT', [tag_virtual],  True)
bT = sqa.index('bT', [tag_virtual],  True)
cT = sqa.index('cT', [tag_virtual],  True)
dT = sqa.index('dT', [tag_virtual],  True)

As for the Hamiltonian:

In [10]:
tensor   =sqa.tensor('Tijab', [aT,bT,iT,jT], [])
operator =sqa.sfExOp([aT,bT,iT,jT])
T        =sqa.term(  1.0, [''], [tensor, operator])

tensor2  =sqa.tensor('Tijab', [cT,dT,kT,lT], [])
operator2=sqa.sfExOp([cT,dT,kT,lT])
T2       =sqa.term(  0.5, [''], [tensor, operator, tensor2, operator2])

### We define the indexes, tensor, operator, and term for $\hat{T}^\dagger$ and ${\hat{T}^\dagger}^2$

We need to define a term that contains an operator acting on the left (on the bra):
$$\left(\hat{E}_{ij}^{ab}\right)^\dagger
=\left(a^\dagger_a a^\dagger_b a_j a_i\right)^\dagger=a^\dagger_i a^\dagger_j a_b a_a=\hat{E}_{ab}^{ij}$$

We need new indexes (note the "T" where before there was "H" in the variable name):

In [11]:
iTd= sqa.index('iTd', [tag_occupied], True)
jTd= sqa.index('jTd', [tag_occupied], True)
kTd= sqa.index('kTd', [tag_occupied], True)
lTd= sqa.index('lTd', [tag_occupied], True)
aTd= sqa.index('aTd', [tag_virtual],  True)
bTd= sqa.index('bTd', [tag_virtual],  True)
cTd= sqa.index('cTd', [tag_virtual],  True)
dTd= sqa.index('dTd', [tag_virtual],  True)

In [12]:
tensor   =sqa.tensor('Tijab', [aTd,bTd,iTd,jTd], [])
operator =sqa.sfExOp([iTd,jTd,aTd,bTd])
Tdagger  =sqa.term( -1.0, [''], [tensor, operator])

tensor2  =sqa.tensor('Tijab', [cTd,dTd,kTd,lTd], [])
operator2=sqa.sfExOp([kTd,lTd,cTd,dTd])
Tdagger2 =sqa.term(  0.5, [''], [tensor, operator, tensor2, operator2])

In [13]:
print(T)
print(T2)
print(Tdagger)
print(Tdagger2)

 (   1.00000)  Tijab[aT,bT,iT,jT] E2[aT,bT,iT,jT] 
 (   0.50000)  Tijab[aT,bT,iT,jT] E2[aT,bT,iT,jT] Tijab[cT,dT,kT,lT] E2[cT,dT,kT,lT] 
 (  -1.00000)  Tijab[aTd,bTd,iTd,jTd] E2[iTd,jTd,aTd,bTd] 
 (   0.50000)  Tijab[aTd,bTd,iTd,jTd] E2[iTd,jTd,aTd,bTd] Tijab[cTd,dTd,kTd,lTd] E2[kTd,lTd,cTd,dTd] 


***

## Define the similarity transformed Hamitonian

Remember that the similarity transform Hamiltonian is used (since $\hat{T}$ and $\hat{H}$ are two-body operator, we can truncate the Taylor expansion to second-order):
$$\begin{array}{rl}
e^{-\hat{T}}\hat{H}e^{\hat{T}}
&=\left(1-\hat{T}^\dagger+\frac{1}{2}{\hat{T}^\dagger}^2\right)
\hat{H}
\left(1+\hat{T}+\frac{1}{2}\hat{T}^2\right)
\\&=
\hat{H}
+\hat{H}\hat{T}
+\frac{1}{2}\hat{H}\hat{T}^2
-\hat{T}^\dagger\hat{H}
-\hat{T}^\dagger\hat{H}\hat{T}
-\frac{1}{2}\hat{T}^\dagger\hat{H}\hat{T}^2
+\frac{1}{2}{\hat{T}^\dagger}^2\hat{H}
+\frac{1}{2}{\hat{T}^\dagger}^2\hat{H}\hat{T}
+\frac{1}{4}{\hat{T}^\dagger}^2\hat{H}\hat{T}^2
\end{array}
\qquad\qquad(C)$$

We define a unit operator (this is not strictly necessary, but we do this to later be able to track down where different terms arose from):

In [14]:
oneleft =sqa.term( 1.0, [''], [sqa.tensor('ONEleft' ,[],[])])
oneright=sqa.term( 1.0, [''], [sqa.tensor('ONEright',[],[])])

In [15]:
print(oneleft)
print(oneright)

 (   1.00000)  ONEleft[] 
 (   1.00000)  ONEright[] 


We now define the similarity-transformed Hamiltonian (we will organize the similarity-transformed terms in a list of length 9 for each term of the expansion in equation (C)):

In [16]:
similH=[]
dict=[]
for left in [oneleft,Tdagger,Tdagger2]:
  for right in [oneright,T,T2]:
    similH.append([])
    dict.append([left,right])
    for elt in H:
      similH[-1]+=[sqa.multiplyTerms(sqa.multiplyTerms(left, elt),right)]
  #break

Each element of the list is a list of length 20 corresponding to the 20 terms of the Hamiltonian:

In [17]:
print(len(similH),[len(elt) for elt in similH])

(9, [20, 20, 20, 20, 20, 20, 20, 20, 20])


***

## Normal-ordered CC equations

Remember that we wish to manipulate
$$C_\mu=\left\langle 0\middle| \hat{E}_\mu \; e^{-\hat{T}}\hat{H}e^{\hat{T}} \middle| 0\right\rangle$$

We define the excitation operator $\hat{E}_\mu$ on which the CC equations are projected. Notice that we give it a tensor $C_\mu$: this is a container that will hold the results of the eventual tensor contraction arising from the manipulation $\left\langle 0\middle| \hat{E}_\mu \; e^{-\hat{T}}\hat{H}e^{\hat{T}} \middle| 0\right\rangle$, so that what needs to be solved is $C_\mu=0$:

In [18]:
iE = sqa.index('iT', [tag_occupied], True)
jE = sqa.index('jT', [tag_occupied], True)
aE = sqa.index('aT', [tag_virtual],  True)
bE = sqa.index('bT', [tag_virtual],  True)
tensor  =sqa.tensor('Cmu', [aE,bE,iE,jE], [])
operator=sqa.sfExOp([iE,jE,aE,bE])
Eijab = sqa.term(1.0, [''], [tensor,operator])

We can observe the expectation value $\left\langle 0\middle| \hat{E}_\mu \; e^{-\hat{T}}\hat{H}e^{\hat{T}} \middle| 0\right\rangle$, for each of the 9x20 terms. Most of them do not survive between the HF determinants $\left\langle 0 \middle|...\middle|0\right\rangle$ : the string of destruction and creation operator mustn't change the occupation pattern of occupied and virtual orbitals.

Let's consider for example the term of the expansion $\left\langle 0\middle| \hat{E}_{ab}^{ij} \; 1.\hat{H}.1 \middle| 0\right\rangle$. The operator on which the equation is projected destroys two virtual orbitals and creates two occupied orbitals (let's write: +2o-2v). To have a non-zero expectation value $\left\langle 0\middle| ... \middle| 0\right\rangle$, the surviving Hamiltonian terms must create two virtual orbitals and destroy two occupied orbitals (i.e.: -2o+2v). Hence only one term, containing $Vvvoo[a,b,i,j]$, survives.

In [19]:
print 'i <.......expectation value.......> list_of_operators        non_zero'
print '---------------------------------------------------------------------'
for i in range(9):
  for elt in similH[i]:
    print '%1i <Tijab.%-11s.H.%-11s> %-24s %-5s'\
      %(i+1,list_tensors(dict[i][0]),\
            list_tensors(dict[i][1]),\
            list_indexes([Eijab,elt]),\
            is_non_zero([Eijab,elt]))

i <.......expectation value.......> list_of_operators        non_zero
---------------------------------------------------------------------
1 <Tijab.ONEleft    .H.ONEright   > oovvoo                   False
1 <Tijab.ONEleft    .H.ONEright   > oovvov                   False
1 <Tijab.ONEleft    .H.ONEright   > oovvvo                   False
1 <Tijab.ONEleft    .H.ONEright   > oovvvv                   False
1 <Tijab.ONEleft    .H.ONEright   > oovvoooo                 False
1 <Tijab.ONEleft    .H.ONEright   > oovvooov                 False
1 <Tijab.ONEleft    .H.ONEright   > oovvoovo                 False
1 <Tijab.ONEleft    .H.ONEright   > oovvoovv                 False
1 <Tijab.ONEleft    .H.ONEright   > oovvovoo                 False
1 <Tijab.ONEleft    .H.ONEright   > oovvovov                 False
1 <Tijab.ONEleft    .H.ONEright   > oovvovvo                 False
1 <Tijab.ONEleft    .H.ONEright   > oovvovvv                 False
1 <Tijab.ONEleft    .H.ONEright   > oovvvooo            

And then, as said before, we normal-order surviving terms of the expectation value $\left\langle 0\middle| \hat{E}_\mu \; e^{-\hat{T}}\hat{H}e^{\hat{T}} \middle| 0\right\rangle$:

**This may take some time, so we'll skip the 5th of 8th element of the `similH` list, which are the most time-consuming**

In [20]:
equ=[]
for i in range(8):
  equ.append([])
  if i!=5 and i!=8: # This is how we skip some terms of the expansion
   for elt in similH[i]:
    if is_non_zero([Eijab,elt]):
      equ[-1]+=sqa.normalOrder(sqa.multiplyTerms(Eijab,elt))
print(len(equ),[len(elt) for elt in equ])

(8, [7, 576, 1657, 0, 1657, 0, 0, 0])


This list of tensor contractions is huge and contains a lot of redundancies and delta function that could be resolved, we hence run the `simplify_all` routine, that basically just ensures there is the least amount of redundancies in the terms (see below for some step-by-step explanations of some terms):

In [21]:
iD  = sqa.index('iD', [tag_occupied], True)
jD  = sqa.index('jD', [tag_occupied], True)
aD  = sqa.index('aD', [tag_virtual],  True)
bD  = sqa.index('bD', [tag_virtual],  True)
deltaO = sqa.tensor('deltaO', [iD,jD], [])
deltaV = sqa.tensor('deltaV', [aD,bD], [])
flatten_equ=[inner for outer in equ for inner in outer]
equ=simplify_all(flatten_equ, [deltaO, deltaO,deltaV])
print(len(equ))

928


Without using symmetry (see next section), this is still too large a number of tensor contraction to _look at_, but it's manageable for a tensor contraction program.

Here is a little script to format the results of the manipulation in a somewhat human-readable format like "$C_\mu=\text{[tensor contractions]}$" :

In [22]:
for i in range(len(equ)):
    elt=equ[i]
    line=''
    for t in elt.tensors:
      if t.name=='Cmu':
        header='%-11s = %4.1f '%(t,elt.numConstant)    
      else:
        line+='%-14s.'%(t)
    print header+line[:-1]
    if i>50:
      print 'etc...'
      break

Cmu[a,b,c,d] =  2.0 ONEleft[]     .ONEright[]    .Vvvoo[a,b,c,d]
Cmu[a,b,c,d] = -1.0 ONEleft[]     .ONEright[]    .Vvvoo[a,b,d,c]
Cmu[a,b,c,d] = -1.0 ONEleft[]     .ONEright[]    .Vvvoo[b,a,c,d]
Cmu[a,b,c,d] =  2.0 ONEleft[]     .ONEright[]    .Vvvoo[b,a,d,c]
Cmu[a,c,b,d] = -8.0 ONEleft[]     .Tijab[a,e,b,f].Voo[b,g]      .deltaV[c,a]   .deltaO[d,b]   .deltaV[e,a]   .deltaO[f,b]   .deltaO[g,b]   
Cmu[a,d,b,e] =  8.0 ONEleft[]     .Tijab[a,f,b,g].Voo[c,h]      .deltaV[d,a]   .deltaO[e,b]   .deltaV[f,a]   .deltaO[g,b]   .deltaO[h,c]   
Cmu[a,d,b,c] =  2.0 ONEleft[]     .Tijab[a,e,b,c].Voo[b,f]      .deltaV[d,a]   .deltaV[e,a]   .deltaO[f,b]   
Cmu[a,d,b,c] =  2.0 ONEleft[]     .Tijab[a,e,b,c].Voo[c,f]      .deltaV[d,a]   .deltaV[e,a]   .deltaO[f,c]   
Cmu[a,e,b,c] = -4.0 ONEleft[]     .Tijab[a,f,b,c].Voo[d,g]      .deltaV[e,a]   .deltaV[f,a]   .deltaO[g,d]   
Cmu[a,b,c,d] =  4.0 ONEleft[]     .Tijab[a,b,c,e].Voo[c,f]      .deltaO[d,c]   .deltaO[e,c]   .deltaO[f,c]   
Cmu[a,b,c,e] = -4.0 

***

## __(ACTIVITY)__ Using symmetry

There is a lot of terms in the final list of tensor contraction operations, but we can reduce the number of terms by indicating the **symmetries** of the tensors provided to `sqa`. 

This is done by the `symmetry` object in `sqa`, which is a rule that takes a sequence of transformation of indexes and a factor for the permutation.

(See for example lines 21-33 of the `sqaSymmetry.py` file for more context.)

This means that to ensure that `sqa` is aware that for example
$$A[j,i]=1.0*A[i,j]$$
the following symmetry object is given when defining the tensor (`twosym` sequence of transformation is `(1,0)`):

In [23]:
tensor=sqa.tensor("A",[iH,jH],[sqa.symmetry((1,0)     , 1)])

Similarly to ensure that `sqa` is aware that for example
$$B[b,a,d,c]=1.0*B[a,b,c,d]$$
the following symmetry object is given when defining the tensor:

In [24]:
tensor=sqa.tensor("B",[aH,bH,cH,dH],[sqa.symmetry((1,0, 3,2), 1)])

**(ACTIVITY)** Try to find out the symmetries of the two-body integral tensors.

**We can hence rerun the notebook** after having replace the tensor definitions by the following:

In [25]:
twosym   = sqa.symmetry((1,0)     , 1)
foursym_1= sqa.symmetry((1,0, 3,2), 1)
foursym_2= sqa.symmetry((0,3, 2,1), 1)
foursym_3= sqa.symmetry((2,1, 0,3), 1)

In [26]:
Voo  =sqa.tensor('Voo',   [iH,jH]      , [twosym]) # this is Voo(i,j)
Vov  =sqa.tensor('Vov',   [iH,aH]      , [      ])
Vvo  =sqa.tensor('Vvo',   [aH,iH]      , [      ])
Vvv  =sqa.tensor('Vvv',   [aH,bH]      , [twosym])
Voooo=sqa.tensor('Voooo', [iH,jH,kH,lH], [foursym_1,foursym_2,foursym_3]) # this is Voooo(i,j,k,l)
Vooov=sqa.tensor('Vooov', [iH,jH,kH,aH], [                    foursym_3])
Voovo=sqa.tensor('Voovo', [iH,jH,aH,kH], [          foursym_2          ])
Voovv=sqa.tensor('Voovv', [iH,jH,aH,bH], [foursym_1                    ])
Vovoo=sqa.tensor('Vovoo', [iH,aH,jH,kH], [                    foursym_3])
Vovov=sqa.tensor('Vovov', [iH,aH,jH,bH], [          foursym_2,foursym_3])
Vovvo=sqa.tensor('Vovvo', [iH,aH,bH,jH], [                             ])
Vovvv=sqa.tensor('Vovvv', [iH,aH,bH,cH], [          foursym_2          ])
Vvooo=sqa.tensor('Vvooo', [aH,iH,jH,kH], [          foursym_2          ])
Vvoov=sqa.tensor('Vvoov', [aH,iH,jH,bH], [                             ])
Vvovo=sqa.tensor('Vvovo', [aH,iH,bH,jH], [          foursym_2,foursym_3])
Vvovv=sqa.tensor('Vvovv', [aH,iH,bH,cH], [                    foursym_3])
Vvvoo=sqa.tensor('Vvvoo', [aH,bH,iH,jH], [foursym_1                    ])
Vvvov=sqa.tensor('Vvvov', [aH,bH,iH,cH], [          foursym_2          ])
Vvvvo=sqa.tensor('Vvvvo', [aH,bH,cH,iH], [                    foursym_3])
Vvvvv=sqa.tensor('Vvvvv', [aH,bH,cH,dH], [foursym_1,foursym_2,foursym_3])

Think also about replacing the symmetry of the amplitudes, `Cmu` container and `delta` functions:

In [27]:
tensor   =sqa.tensor('Tijab', [aT,bT,iT,jT], [foursym_1])
operator =sqa.sfExOp([aT,bT,iT,jT])
T        =sqa.term(  1.0, [''], [tensor, operator])

tensor2  =sqa.tensor('Tijab', [cT,dT,kT,lT], [foursym_1])
operator2=sqa.sfExOp([cT,dT,kT,lT])
T2       =sqa.term(  0.5, [''], [tensor, operator, tensor2, operator2])

In [28]:
tensor   =sqa.tensor('Tijab', [aTd,bTd,iTd,jTd], [foursym_1])
operator =sqa.sfExOp([iTd,jTd,aTd,bTd])
Tdagger  =sqa.term( -1.0, [''], [tensor, operator])

tensor2  =sqa.tensor('Tijab', [cTd,dTd,kTd,lTd], [foursym_1])
operator2=sqa.sfExOp([kTd,lTd,cTd,dTd])
Tdagger2 =sqa.term(  0.5, [''], [tensor, operator, tensor2, operator2])

In [29]:
tensor   =sqa.tensor('Cmu', [aE,bE,iE,jE], [foursym_1])
operator =sqa.sfExOp([iE,jE,aE,bE])
Eijab    = sqa.term( 1.0, [''], [tensor,operator])

In [30]:
deltaO = sqa.tensor('deltaO', [iD,jD], [twosym])
deltaV = sqa.tensor('deltaV', [aD,bD], [twosym])

### RESERVOIR

Here the key cells that would be modified when running the notebook with or without symmetry are repeated, so as to prevent loss of information

#### WITHOUT SYMMETRY

In [31]:
Voo  =sqa.tensor('Voo',   [iH,jH]      , []) # this is Voo(i,j)
Vov  =sqa.tensor('Vov',   [iH,aH]      , [])
Vvo  =sqa.tensor('Vvo',   [aH,iH]      , [])
Vvv  =sqa.tensor('Vvv',   [aH,bH]      , [])
Voooo=sqa.tensor('Voooo', [iH,jH,kH,lH], []) # this is Voooo(i,j,k,l)
Vooov=sqa.tensor('Vooov', [iH,jH,kH,aH], [])
Voovo=sqa.tensor('Voovo', [iH,jH,aH,kH], [])
Voovv=sqa.tensor('Voovv', [iH,jH,aH,bH], [])
Vovoo=sqa.tensor('Vovoo', [iH,aH,jH,kH], [])
Vovov=sqa.tensor('Vovov', [iH,aH,jH,bH], [])
Vovvo=sqa.tensor('Vovvo', [iH,aH,bH,jH], [])
Vovvv=sqa.tensor('Vovvv', [iH,aH,bH,cH], [])
Vvooo=sqa.tensor('Vvooo', [aH,iH,jH,kH], [])
Vvoov=sqa.tensor('Vvoov', [aH,iH,jH,bH], [])
Vvovo=sqa.tensor('Vvovo', [aH,iH,bH,jH], [])
Vvovv=sqa.tensor('Vvovv', [aH,iH,bH,cH], [])
Vvvoo=sqa.tensor('Vvvoo', [aH,bH,iH,jH], [])
Vvvov=sqa.tensor('Vvvov', [aH,bH,iH,cH], [])
Vvvvo=sqa.tensor('Vvvvo', [aH,bH,cH,iH], [])
Vvvvv=sqa.tensor('Vvvvv', [aH,bH,cH,dH], [])

In [32]:
tensor   =sqa.tensor('Tijab', [aT,bT,iT,jT], [])
operator =sqa.sfExOp([aT,bT,iT,jT])
T        =sqa.term(  1.0, [''], [tensor, operator])

tensor2  =sqa.tensor('Tijab', [cT,dT,kT,lT], [])
operator2=sqa.sfExOp([cT,dT,kT,lT])
T2       =sqa.term(  0.5, [''], [tensor, operator, tensor2, operator2])

In [33]:
tensor   =sqa.tensor('Tijab', [aTd,bTd,iTd,jTd], [])
operator =sqa.sfExOp([iTd,jTd,aTd,bTd])
Tdagger  =sqa.term( -1.0, [''], [tensor, operator])

tensor2  =sqa.tensor('Tijab', [cTd,dTd,kTd,lTd], [])
operator2=sqa.sfExOp([kTd,lTd,cTd,dTd])
Tdagger2 =sqa.term(  0.5, [''], [tensor, operator, tensor2, operator2])

In [34]:
iE = sqa.index('iT', [tag_occupied], True)
jE = sqa.index('jT', [tag_occupied], True)
aE = sqa.index('aT', [tag_virtual],  True)
bE = sqa.index('bT', [tag_virtual],  True)
tensor   =sqa.tensor('Cmu', [aE,bE,iE,jE], [])
operator =sqa.sfExOp([iE,jE,aE,bE])
Eijab    =sqa.term( 1.0, [''], [tensor,operator])

In [35]:
iD  = sqa.index('iD', [tag_occupied], True)
jD  = sqa.index('jD', [tag_occupied], True)
aD  = sqa.index('aD', [tag_virtual],  True)
bD  = sqa.index('bD', [tag_virtual],  True)
deltaO = sqa.tensor('deltaO', [iD,jD], [])
deltaV = sqa.tensor('deltaV', [aD,bD], [])
#equ=simplify_all(equ,[deltaO, deltaO,deltaV])
#print(len(equ))

#### WITH SYMMETRY

In [36]:
Voo  =sqa.tensor('Voo',   [iH,jH]      , [twosym]) # this is Voo(i,j)
Vov  =sqa.tensor('Vov',   [iH,aH]      , [      ])
Vvo  =sqa.tensor('Vvo',   [aH,iH]      , [      ])
Vvv  =sqa.tensor('Vvv',   [aH,bH]      , [twosym])
Voooo=sqa.tensor('Voooo', [iH,jH,kH,lH], [foursym_1,foursym_2,foursym_3]) # this is Voooo(i,j,k,l)
Vooov=sqa.tensor('Vooov', [iH,jH,kH,aH], [                    foursym_3])
Voovo=sqa.tensor('Voovo', [iH,jH,aH,kH], [          foursym_2          ])
Voovv=sqa.tensor('Voovv', [iH,jH,aH,bH], [foursym_1                    ])
Vovoo=sqa.tensor('Vovoo', [iH,aH,jH,kH], [                    foursym_3])
Vovov=sqa.tensor('Vovov', [iH,aH,jH,bH], [          foursym_2,foursym_3])
Vovvo=sqa.tensor('Vovvo', [iH,aH,bH,jH], [                             ])
Vovvv=sqa.tensor('Vovvv', [iH,aH,bH,cH], [          foursym_2          ])
Vvooo=sqa.tensor('Vvooo', [aH,iH,jH,kH], [          foursym_2          ])
Vvoov=sqa.tensor('Vvoov', [aH,iH,jH,bH], [                             ])
Vvovo=sqa.tensor('Vvovo', [aH,iH,bH,jH], [          foursym_2,foursym_3])
Vvovv=sqa.tensor('Vvovv', [aH,iH,bH,cH], [                    foursym_3])
Vvvoo=sqa.tensor('Vvvoo', [aH,bH,iH,jH], [foursym_1                    ])
Vvvov=sqa.tensor('Vvvov', [aH,bH,iH,cH], [          foursym_2          ])
Vvvvo=sqa.tensor('Vvvvo', [aH,bH,cH,iH], [                    foursym_3])
Vvvvv=sqa.tensor('Vvvvv', [aH,bH,cH,dH], [foursym_1,foursym_2,foursym_3])

In [37]:
tensor   =sqa.tensor('Tijab', [aT,bT,iT,jT], [foursym_1])
operator =sqa.sfExOp([aT,bT,iT,jT])
T        =sqa.term(  1.0, [''], [tensor, operator])

tensor2  =sqa.tensor('Tijab', [cT,dT,kT,lT], [foursym_1])
operator2=sqa.sfExOp([cT,dT,kT,lT])
T2       =sqa.term(  0.5, [''], [tensor, operator, tensor2, operator2])

In [38]:
tensor   =sqa.tensor('Tijab', [aTd,bTd,iTd,jTd], [])
operator =sqa.sfExOp([iTd,jTd,aTd,bTd])
Tdagger  =sqa.term( -1.0, [''], [tensor, operator])

tensor2  =sqa.tensor('Tijab', [cTd,dTd,kTd,lTd], [])
operator2=sqa.sfExOp([kTd,lTd,cTd,dTd])
Tdagger2 =sqa.term(  0.5, [''], [tensor, operator, tensor2, operator2])

In [39]:
iD  = sqa.index('iD', [tag_occupied], True)
jD  = sqa.index('jD', [tag_occupied], True)
aD  = sqa.index('aD', [tag_virtual],  True)
bD  = sqa.index('bD', [tag_virtual],  True)
deltaO = sqa.tensor('deltaO', [iD,jD], [twosym])
deltaV = sqa.tensor('deltaV', [aD,bD], [twosym])
#equ=simplify_all(equ,[deltaO, deltaO,deltaV])
#print(len(equ))

## **(ACTIVITY)** Redundancies

There is a lot of redundancies built in this script (definition of the tensors, of the operators, etc...). For example the operators of the type $\hat{E}_{cc}^{ca}$ and of the type $\hat{E}_{cc}^{ac}$ are equivalent, etc...

Find a way to construct the Hamiltonian that detects and prevents these redundancies.